In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('ner_datasetreference.csv',encoding='latin1',dtype=str)
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [3]:
df['Sentence #']=df['Sentence #'].fillna(method='ffill')
sentences=[]
ner_tags=[]

C:\Users\91637\AppData\Local\Temp\ipykernel_45756\3954636914.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Sentence #']=df['Sentence #'].fillna(method='ffill')


In [4]:
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [5]:
for _, group in df.groupby('Sentence #'):
    sentences.append(group['Word'].tolist())
    ner_tags.append(group['Tag'].tolist())
len(sentences),len(ner_tags)


(47959, 47959)

In [6]:
from datasets import Dataset,DatasetDict,Features,Sequence,Value
features=Features(
    {
        'tokens':Sequence(Value('string')),
        'ner_tags':Sequence(Value('string'))
    }
)
hf=Dataset.from_dict(
    {
        'tokens':sentences,
        'ner_tags':ner_tags
    },
    features=features
)
dataset=DatasetDict({'train':hf})
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 47959
    })
})

In [7]:
# dataset['train'][0]

In [8]:
unique_tags=sorted({tag for seq in ner_tags for tag in seq})
tag2id={tag: i for i , tag in enumerate(unique_tags)}
id2tag={i: tag for tag, i in tag2id.items()}

# tag2id

In [9]:
from transformers import AutoTokenizer,AutoModelForTokenClassification

model='distilbert-base-cased'

tokenizer=AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [10]:
mod=AutoModelForTokenClassification.from_pretrained(
    model,
    num_labels=len(unique_tags),
    id2label=id2tag,
    label2id=tag2id
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def tokenize_and_align_labels(batch):
    tokenized=tokenizer(
        batch['tokens'],
        is_split_into_words=True,
        truncation=True,
        padding='max_length',
        max_length=128
    )
    aligned=[]
    for i,tags in enumerate(batch['ner_tags']):
        word_ids=tokenized.word_ids(batch_index=i)
        label_ids=[]
        previous=None

        for w_id in word_ids:
            if w_id is None:
                label_ids.append(-100)
            elif w_id != previous:
                label_ids.append(tag2id[tags[w_id]])
            else:
                label_ids.append(-100)
            previous=w_id
        aligned.append(label_ids)
    tokenized['labels']=aligned
    return tokenized

tokenized_ds=dataset.map(tokenize_and_align_labels,batched=True)

Map:   0%|          | 0/47959 [00:00<?, ? examples/s]

In [12]:
# tokenized_ds["train"][0]
print(type(dataset["train"][0]["tokens"]))
print(type(dataset["train"][0]["ner_tags"]))



<class 'list'>
<class 'list'>


In [13]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)


In [14]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir="ner-model",
    per_device_train_batch_size=8,
    num_train_epochs=2,
    logging_steps=50,
    learning_rate=5e-5
)

trainer = Trainer(
    model=mod,
    args=args,
    train_dataset=tokenized_ds["train"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()


C:\Users\91637\AppData\Local\Temp\ipykernel_45756\414218658.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,0.588900
100,0.231000
150,0.184000
200,0.188600
250,0.150300
300,0.150000
350,0.164300
400,0.151000
450,0.156300
500,0.139400


TrainOutput(global_step=11990, training_loss=0.08731297370888373, metrics={'train_runtime': 2052.7929, 'train_samples_per_second': 46.726, 'train_steps_per_second': 5.841, 'total_flos': 3133845738269184.0, 'train_loss': 0.08731297370888373, 'epoch': 2.0})

In [32]:
model_path = "ner-model/checkpoint-11990"


In [34]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_path = "ner-model/checkpoint-11990"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)

model.eval()


DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
   

In [35]:
import torch

def ner_predict(text):
    words = text.split()
    inputs = tokenizer(words, is_split_into_words=True, return_tensors="pt")

    with torch.no_grad():
        logits = model(**inputs).logits

    pred_ids = logits.argmax(-1)[0].tolist()
    id2label = model.config.id2label

    results = []
    for token, pred, word_id in zip(inputs.tokens(), pred_ids, inputs.word_ids()):
        if word_id is not None:
            results.append((words[word_id], id2label[pred]))

    return results


In [36]:
ner_predict("Barack Obama lives in New York")


[('Barack', 'B-per'),
 ('Obama', 'I-per'),
 ('lives', 'O'),
 ('in', 'O'),
 ('New', 'B-geo'),
 ('York', 'I-geo')]

In [ ]:
ner_predict('OpenAI ')